In [1]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import key_param

/home/msp/Documents/RAG/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = MongoClient(key_param.MONGO_URI)
dbName = "PKU"
collectionName = "collection_of_pku_text_blobs"
collection = client[dbName][collectionName]

In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=key_param.openai_api_key)

vectorStore = MongoDBAtlasVectorSearch(collection, embeddings)

In [4]:
def query_data(query):
    docs = vectorStore.similarity_search(query, K=1)
    as_output = docs[0].page_content

    llm = OpenAI(openai_api_key=key_param.openai_api_key, temperature=0)
    retriever = vectorStore.as_retriever()
    qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)
    retriever_output = qa.run(query)

    return as_output, retriever_output

In [10]:
embedded_query = embeddings.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

[0.005354681365594308,
 -0.0005715346531097276,
 0.038875909934336914,
 -0.0029596003572924627,
 -0.008966285328704282]

In [11]:
len(embedded_query)

1536

In [7]:
docs = vectorStore.similarity_search("Can you describe PKU?", k=2)
docs

[]